In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error

# ----------------------------
# 1. Simulate Data
# ----------------------------
np.random.seed(42)
dates = pd.date_range(start="2023-01-01", periods=120, freq="D")

def generate_series(base, noise=0.1):
    return base + np.sin(np.arange(120)/5) + np.random.normal(0, noise, 120)

data = {
    "Cars_Distributed": generate_series(5),
    "Cash_Incentive": generate_series(10)
}

# ----------------------------
# 2. Create Sequences
# ----------------------------
def create_sequences(series, seq_length):
    X, y = [], []
    for i in range(len(series) - seq_length):
        X.append(series[i:i+seq_length])
        y.append(series[i+seq_length])
    return np.array(X), np.array(y)

SEQ_LEN = 20
X_car, y_car = create_sequences(data["Cars_Distributed"], SEQ_LEN)
X_cash, y_cash = create_sequences(data["Cash_Incentive"], SEQ_LEN)

# Use Cars_Distributed as example
X = X_car
y = y_car

# Convert to torch tensors
X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)  # (batch, seq_len, features=1)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)

dataset = TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

# ----------------------------
# 3. Define Transformer Model
# ----------------------------
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=2, seq_length=SEQ_LEN):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        self.pos_embedding = nn.Parameter(torch.randn(1, seq_length, d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.input_proj(x) + self.pos_embedding
        x = self.transformer(x)
        out = self.fc_out(x[:, -1, :])  # take last time step
        return out

model = TimeSeriesTransformer()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# ----------------------------
# 4. Train Model
# ----------------------------
EPOCHS = 50
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for xb, yb in loader:
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {epoch_loss/len(loader):.4f}")

# ----------------------------
# 5. Forecast (Next 20 days)
# ----------------------------
model.eval()
forecast = []
input_seq = X_tensor[-1].unsqueeze(0)  # take last sequence

for _ in range(20):
    with torch.no_grad():
        pred = model(input_seq)
        forecast.append(pred.item())
        # shift sequence and append prediction
        input_seq = torch.cat([input_seq[:,1:,:], pred.unsqueeze(-1)], dim=1)

forecast_dates = pd.date_range(start=dates[len(dates)-1]+pd.Timedelta(days=1), periods=20)

# ----------------------------
# 6. Plot Results
# ----------------------------
plt.figure(figsize=(12,5))
plt.plot(dates, data["Cars_Distributed"], label="Actual")
plt.plot(forecast_dates, forecast, label="Forecast", color="r")
plt.title("Cars Distributed Forecast (Attention-based Transformer)")
plt.xlabel("Date")
plt.ylabel("Cars Distributed")
plt.legend()
plt.show()

# ----------------------------
# 7. Evaluation (on training last 20 points)
# ----------------------------
y_true = data["Cars_Distributed"][-20:]
y_pred = forecast

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)

print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}")